# SB1 Big Data Downloads: I-405 Skirball
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [1]:
import pandas as pd
from siuba import *
import ast


from calitp_data_analysis.sql import to_snakecase

import altair as alt
from calitp_data_analysis import calitp_color_palette as cp

from IPython.display import Markdown, HTML, display_html, display
from IPython.core.display import display

import numpy as np

/tmp/ipykernel_516/2202862553.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
from dla_utils import _dla_utils

In [3]:
import _utils

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)

In [5]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

### Read in Replica Data Spring 2023

In [6]:
replica_trips = "replica-405_skirball-05_24_24-trips_dataset.csv"

In [7]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_516/2643890524.py:1: DtypeWarning: Columns (18,19,20,25,26,28,29,30,31,33,36,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [8]:
display(HTML("<h4><strong> Sample of data</strong></h4>"))


In [9]:
r_trips.sample(5)

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
257013,9302896571626397264,"2 (Tract 2169.02, Los Angeles, CA)","2169.02 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 1281.02, Los Angeles, CA)","1281.02 (Los Angeles, CA)","Los Angeles County, CA",California,private_auto,home,social,12:17:00,12:50:00,33,18.90,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,single_family,single_family,multi_family,multi_family,7579849294298285797,9607203881269597076,33.00,male,white_not_hispanic_or_latino,employed,in_person,"59,218.00",private_auto,3.00,"184,761.00",two,core,naics23,several_units,not_attending_school,some_college,renter,english,"1 (Tract 1281.02, Los Angeles, CA)","1281.02 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 2171.01, Los Angeles, CA)","2171.01 (Los Angeles, CA)","Los Angeles County, CA",California
366384,10044791123655181857,"2 (Tract 5033.01, Los Angeles, CA)","5033.01 (Los Angeles, CA)","Los Angeles County, CA",California,"2 (Tract 2622, Los Angeles, CA)","2622 (Los Angeles, CA)","Los Angeles County, CA",California,auto_passenger,home,social,12:55:16,13:57:55,62,35.70,unknown_vehicle_type,unknown_fuel_type,NaN,NaN,NaN,single_family,single_family,single_family,single_family,1348767672027219422,14444669387201913369,27.00,female,black_not_hispanic_or_latino,employed,in_person,"94,986.00",private_auto,2.00,"94,986.00",three_plus,core,naics61,single_family,not_attending_school,advanced_degree,owner,english,"2 (Tract 2622, Los Angeles, CA)","2622 (Los Angeles, CA)","Los Angeles County, CA",California,"2 (Tract 1397.01, Los Angeles, CA)","1397.01 (Los Angeles, CA)","Los Angeles County, CA",California
404087,8025058747789155362,"1 (Tract 2623.03, Los Angeles, CA)","2623.03 (Los Angeles, CA)","Los Angeles County, CA",California,"2 (Tract 1041.03, Los Angeles, CA)","1041.03 (Los Angeles, CA)","Los Angeles County, CA",California,private_auto,home,social,13:39:16,14:19:59,40,21.00,unknown_vehicle_type,other_non_bev,NaN,NaN,NaN,single_family,single_family,single_family,single_family,15874717675890836160,5139820635381915520,52.00,female,hispanic_or_latino_origin,not_in_labor_force,unemployed_under_16_not_in_labor_force,0.00,other_travel_mode,13.00,"115,058.00",three_plus,core,not_working,single_family,not_attending_school,k_12,owner,spanish,"2 (Tract 1041.03, Los Angeles, CA)","1041.03 (Los Angeles, CA)","Los Angeles County, CA",California,Does not have work/school location,Does not have work/school location,Does not have work/school location,Does not have work/school location
213866,7281458345523096603,"1 (Tract 1375.01, Los Angeles, CA)","1375.01 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 7014.02, Los Angeles, CA)","7014.02 (Los Angeles, CA)","Los Angeles County, CA",California,auto_passenger,work,home,06:32:00,07:14:10,42,21.40,unknown_vehicle_type,unknown_fuel_type,NaN,NaN,NaN,single_family,single_family,mixed_use,healthcare,6116471627154071969,7757

In [10]:
display(HTML("<h4><strong> Columns in Replica Trips Data</strong></h4>"))


In [11]:
r_trips.columns

Index(['activity_id', 'origin_bgrp_2020', 'origin_trct_2020',
       'origin_cty_2020', 'origin_st_2020', 'destination_bgrp_2020',
       'destination_trct_2020', 'destination_cty_2020', 'destination_st_2020',
       'primary_mode', 'trip_purpose', 'previous_trip_purpose',
       'trip_start_time', 'trip_end_time', 'trip_duration_minutes',
       'trip_distance_miles', 'vehicle_type', 'vehicle_fuel_type',
       'transit_submode', 'transit_agency', 'transit_route', 'origin_land_use',
       'origin_building_use', 'destination_land_use',
       'destination_building_use', 'trip_taker_person_id',
       'trip_taker_household_id', 'trip_taker_age', 'trip_taker_sex',
       'trip_taker_race_ethnicity', 'trip_taker_employment_status',
       'trip_taker_wfh', 'trip_taker_individual_income',
       'trip_taker_commute_mode', 'trip_taker_household_size',
       'trip_taker_household_income', 'trip_taker_available_vehicles',
       'trip_taker_resident_type', 'trip_taker_industry',
       'tri

#### Data Exploration: Replica Trips
* Summarizing the data that we exported from Replica
* Existing visualizations in replica: 
   * Primary Mode
   * Trip Purpose
   * Starting hour 
   * Trip Duration/Distance
   * Origin Destination
   * Vehicle Fuel type
   * Transit Routes/Stops/Sub mode/Agency
   * Household Income
   * Race and Ethnicity
   * Private Auto Availability
   * Age
   * Employment/School Status
   
   

In [12]:
# def basic_bar_chart(df, x_col, y_col, color_col):

#     chart = (alt.Chart(df)
#              .mark_bar()
#              .encode(
#                  x=alt.X(x_col, title=labeling(x_col)),
#                  y=alt.Y(y_col, title=labeling(y_col)),
#                  color = (alt.Color(color_col,
#                                   scale=alt.Scale(
#                                       range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
#                                       legend=alt.Legend(title=(labeling(color_col)), symbolLimit=10)
#                                   )),
#                  tooltip=[alt.Tooltip(x_col, title=labeling(x_col)),
#                           alt.Tooltip(y_col, title=labeling(y_col))]
#                                    )
#             )

#     chart=styleguide.preset_chart_config(chart)
#     chart = add_tooltip(chart, labeling(x_col), labeling(y_col))
#     return chart

In [13]:
trips = r_trips>>group_by(_.primary_mode)>>summarize(avg_trip_time = _.trip_duration_minutes.mean(),
                                                     number_trips = _.activity_id.nunique())

In [14]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   primary_mode   5 non-null      object 
 1   avg_trip_time  5 non-null      float64
 2   number_trips   5 non-null      int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 248.0+ bytes


In [15]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("avg_trip_time"),
            color=alt.Color("avg_trip_time", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)
            ), tooltip=trips.columns.tolist())
      .properties(title = "Average Trip Time",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [16]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("number_trips", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,)
            ), tooltip=trips.columns.tolist())
      .properties(title="Number of Trips",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [17]:
display(HTML("<h4><strong>Length of Trips dataframe</strong></h4>"))

In [18]:
len(r_trips)

466756

In [19]:
display(HTML("<h4><strong>Number of unique activity ids in data</strong></h4>"))

In [20]:
r_trips>>summarize(number_trips = _.activity_id.nunique())

,number_trips
0,466756


In [21]:
display(HTML("<h4><strong>Number of unique trip taker ids</strong></h4>")) 

In [22]:
r_trips>>summarize(number_trips = _.trip_taker_person_id.nunique())

,number_trips
0,281798


In [23]:
race_ethnicity_by_mode = r_trips>>group_by(_.primary_mode)>>count(_.trip_taker_race_ethnicity)

In [24]:
display(HTML("<h4><strong>Trip Taker Race and Ethnicity by Mode</strong></h4>")) 

In [25]:
chart = (
        alt.Chart(race_ethnicity_by_mode)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("n"),
            color=alt.Color("trip_taker_race_ethnicity", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=race_ethnicity_by_mode["trip_taker_race_ethnicity"].unique().tolist())
            ),
            tooltip=race_ethnicity_by_mode.columns.tolist())
            
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [26]:
display(HTML("<H4><strong>Transit Mode Splits</strong></H4>")) 


In [27]:
display(HTML("<strong>Looking at transit mode, need to alter the transit_agency col and transit_submode to get a better count.</strong>")) 


In [28]:
# ptt_agency_count = (r_trips
#  >>filter(_.primary_mode=="public_transit")
#  >>group_by(_.primary_mode, _.transit_submode, _.transit_agency)
#  >>summarize(n =_.activity_id.nunique())
#  >>arrange(-_.n))

In [29]:
# ptt_agency_count

In [30]:
display(HTML("<h4><strong>Most common transit mode combinations</strong></h4>")) 


In [31]:
# ptt_modes.head(10)

In [32]:
##### unnesting the transit submode and agencies to get counts. 

In [33]:
(r_trips>>filter(_.primary_mode=="public_transit")).sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
61898,7706952349763026875,"2 (Tract 1111, Los Angeles, CA)","1111 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 2060.51, Los Angeles, CA)","2060.51 (Los Angeles, CA)","Los Angeles County, CA",California,public_transit,work,home,05:35:00,07:53:14,138,29.80,unknown_vehicle_type,unknown_fuel_type,"bus, bus","LADOTMVN, Metro - Los Angeles","CE573, Metro Rapid Line",single_family,single_family,retail,retail,15982840578351068133,16243967259505879865,48.00,male,hispanic_or_latino_origin,employed,in_person,"46,903.00",private_auto,5.00,"88,596.00",zero,core,naics56,single_family,not_attending_school,high_school,renter,spanish,"2 (Tract 1111, Los Angeles, CA)","1111 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 2060.51, Los Angeles, CA)","2060.51 (Los Angeles, CA)","Los Angeles County, CA",California


In [34]:
agencies_test, mode_test = _utils.get_tranist_agency_counts(r_trips, "primary_mode", "transit_submode", "transit_agency", "activity_id")

In [35]:
agencies_test

,primary_mode,transit_submode,transit_agency,n,agency_count,n_modes_taken
15,public_transit,"bus, bus","Metro - Los Angeles, Metro - Los Angeles",501,1,2
57,public_transit,"bus, bus, bus","Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles",375,1,3
27,public_transit,"bus, bus, bus","Big Blue Bus, Metro - Los Angeles, Metro - Los Angeles",192,2,3
5,public_transit,"bus, bus","Big Blue Bus, Metro - Los Angeles",171,2,2
1,public_transit,bus,Metro - Los Angeles,145,1,1
...,...,...,...,...,...,...
118,public_transit,"rail, bus, bus, bus","Amtrak, Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles",1,2,4
119,public_transit,"rail, bus, bus, rail","Amtrak, Big Blue Bus, Metro - Los Angeles, Amtrak",1,3,4
120,public_transit,"rail, bus, light_rail, bus","Amtrak, Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles",1,2,4
121,public_transit,"subway, bus","Metro - Los Angeles, Metro - Los Angeles",1,1,2


In [36]:
mode_test

,transit_submode,n,n_modes_taken
2,"bus, bus, bus",855,3
1,"bus, bus",842,2
0,bus,163,1
8,"bus, light_rail, bus",124,3
3,"bus, bus, bus, bus",85,4
12,"light_rail, bus",56,2
13,"light_rail, bus, bus",24,3
9,"bus, light_rail, bus, bus",18,4
15,"light_rail, light_rail, bus",12,3
6,"bus, bus, subway",7,3


In [37]:
## This line of code gets all agencies listed, even if agency is named twice. 
## Need code that counts the unique agencies
#ptt_agency_count['agency_count'] = ptt_agency_count.transit_agency.apply(lambda x: len(x.split(", ")))

In [38]:
modes_count = agencies_test>>group_by(_.n_modes_taken)>>summarize(n_trips = _.n.sum())

In [39]:
modes_count

,n_modes_taken,n_trips
0,1,163
1,2,901
2,3,1030
3,4,117


In [40]:
chart = (
        alt.Chart((modes_count))
        .mark_bar(size=60)
        .encode(
            x=alt.X("n_modes_taken", title ="Number of Modes Taken per Trip"),
            y=alt.Y("n_trips", title = "Number of Trips"),
            color=alt.Color("n_trips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=modes_count.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [41]:
agency_mode_trips = agencies_test>>group_by(_.agency_count, _.n_modes_taken)>>summarize(ntrips=_.n.sum())>>arrange(-_.ntrips)

In [42]:
chart = (
        alt.Chart((agency_mode_trips))
        .mark_circle(size=100)
        .encode(
            x=alt.X("agency_count", title ="Number of Agencies"),
            y=alt.Y("n_modes_taken", title = "Number of Modes Taken"),
            color=alt.Color("ntrips", title = "Number of Trips",
                            scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS)),
            tooltip=agency_mode_trips.columns.tolist())
            
      .properties(title = "How Many Modes are Taken Per Trip",
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [43]:
##### Getting columns for each agency and counts 

In [44]:
# ptt_agency_count['transit_agency'].unique().tolist()

In [45]:
agency_list = _utils.get_list_of_agencies(agencies_test, "transit_agency")

In [46]:
# len(agency_list)

In [47]:
display(HTML("<h4><strong>Agencies Identified in Trips Data</strong></h4>")) 

(agency_list)

{'Amtrak',
 'Avta',
 'Beach Cities Transit-City Of Redondo Beach',
 'Big Blue Bus',
 'Culver Citybus',
 'Foothill Transit',
 'Gtrans',
 'La Go Bus',
 'Ladot126',
 'Ladotdt',
 'Ladotmvn',
 'Ladotmvs',
 'Long Beach Transit',
 'Metro - Los Angeles',
 'Metrolink Trains',
 'Santa Clarita Transit'}

In [48]:
### making a copy of the ptt_agency_count
# df = r_trips.copy()

##### Trying ChatGPT approach

In [49]:
# agency_indv_count = (df>>filter(_.primary_mode=="public_transit")>>select(_.transit_agency))
# agency_indv_count['transit_agency'] = agency_indv_count['transit_agency'].astype(str)

In [50]:
# agency_indv_count

In [51]:
df = _utils.get_dummies_by_agency(agencies_test, "transit_agency")

In [52]:
##### Identifying trips with one agency

In [53]:
cols_to_keep = ["transit_submode","unique_agencies", "n","n_modes_taken"]

In [54]:
# chart = (alt.Chart(df)
#              .mark_circle(size=100)
#              .encode(
#                  x=alt.X("n_modes_taken", title="Number of Modes taken"),
#                  y=alt.Y("n", title="Number of Trips"),
#                  color = alt.Color("agency_count", title="Number of Unique Agencies",
#                                   scale=alt.Scale(
#                                       range=cp.CALITP_DIVERGING_COLORS,
#                                       domain=df["agency_count"].unique().tolist())
#             ),
#             tooltip=cols_to_keep)
#         .properties(title = ("Transit Trips Agency Breakdown"), width=500,
#     height=300)
#         )
# chart                    

In [55]:
# (df>>filter(_.agency_count==1)>>arrange(-_.n))

In [56]:
chart = (alt.Chart((df>>filter(_.agency_count==1)))
             .mark_bar()
             .encode(
                 x=alt.X("unique_agencies", title="Agency"),
                 y=alt.Y("n", title="Number of Modes taken"),
                 color = alt.Color("n_modes_taken", title="Number of Trips",
                                  scale=alt.Scale(
                                      range=cp.CALITP_SEQUENTIAL_COLORS,)),
            tooltip=cols_to_keep)
        .properties(title = ("Transit Trips With Only One Agency"), width=800,
    height=500)
        )
chart           

alt.Chart(...)

In [57]:
display(HTML("<h4><strong>Most Common Agency Combination</strong></h4>")) 

(df>>arrange(-_.n)>>select(_.transit_agency)).head(5)

,transit_agency
15,"Metro - Los Angeles, Metro - Los Angeles"
57,"Metro - Los Angeles, Metro - Los Angeles, Metro - Los Angeles"
27,"Big Blue Bus, Metro - Los Angeles, Metro - Los Angeles"
5,"Big Blue Bus, Metro - Los Angeles"
1,Metro - Los Angeles


In [58]:
df_agencies = _utils.get_agencies_occurances(df)

/home/jovyan/data-analyses/sb125_analyses/corridor_study/_utils.py:102: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_agencies['n_trips'] = df_agencies[list(df_agencies.columns)].sum(axis=1)


In [59]:
df_agencies

,agency,n_trips
0,Foothill Transit,1
1,LADOTMVS,4
2,LADOTMVN,54
3,AVTA,4
4,Beach Cities Transit-City of Redondo Beach,2
5,Metro - Los Angeles,195
6,Long Beach Transit,1
7,LADOT126,4
8,Santa Clarita Transit,28
9,Metrolink Trains,1


In [60]:
tooltip_cols = ["agency", "n_trips"]

In [61]:
chart = (alt.Chart(df_agencies)
        .mark_bar()
        .encode(
            x=alt.X("agency", title = "Agency Name"),
            y=alt.Y("n_trips", title= "Number of boardings reported for trips (One person taking two LA Metro trips will count as 2)"),
            color=alt.Color("n_trips", scale=alt.Scale(range = cp.CALITP_SEQUENTIAL_COLORS)),
            tooltip = tooltip_cols)
      .properties(title = "Number of Times an Agency was used for Trip Taking",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [62]:
display(HTML("<h4><strong>Trips by Resident Type</strong></h4>")) 


In [63]:
trip_by_res_type = (r_trips
 >>group_by(_.primary_mode,_.trip_taker_resident_type)
 >>summarize(number_trips = _.activity_id.nunique())
 >>arrange(_.primary_mode))

In [64]:
trip_by_res_type

,primary_mode,trip_taker_resident_type,number_trips
0,auto_passenger,core,116685
1,auto_passenger,donut,2
2,auto_passenger,visitor,28818
3,commercial,NaN,10154
4,on_demand_auto,core,3191
5,on_demand_auto,visitor,595
6,private_auto,core,294704
7,private_auto,donut,1
8,private_auto,visitor,10394
9,private_auto,NaN,1


In [65]:
chart = (
        alt.Chart(trip_by_res_type)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode", title = "Mode"),
            y=alt.Y("number_trips", title = "Number of Trips"),
            color=alt.Color("trip_taker_resident_type", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
                                                                         domain=trip_by_res_type["trip_taker_resident_type"].unique().tolist())
            ),
            tooltip=trip_by_res_type.columns.tolist())
      .properties(title = "Trips by Resident Type",
    width=800,
    height=500)
        )
chart 

alt.Chart(...)

In [66]:
display(HTML("<h4><strong>Unique Household Ids</strong></h4>")) 

In [67]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
160790,,33634
267303,NaN,16384
79436,9146817053558612433,16
23626,2709972826117271852,15
30303,3488167635549658463,14
...,...,...
267296,9999130854496671765,1
267299,999930961444594419,1
267300,9999534358623422158,1
267301,9999886348099258237,1


In [68]:
display(HTML("<h4><strong>Checking one household id</strong></h4>")) 


In [69]:
print(len(r_trips>>filter(_.trip_taker_household_id == 9146817053558612433)))
(r_trips>>filter(_.trip_taker_household_id == 9146817053558612433)).sample()

16


,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
304389,11797067000884676673,"1 (Tract 7010, Los Angeles, CA)","7010 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 2623.01, Los Angeles, CA)","2623.01 (Los Angeles, CA)","Los Angeles County, CA",California,auto_passenger,home,work,18:33:00,18:55:47,22,10.00,unknown_vehicle_type,unknown_fuel_type,NaN,NaN,NaN,office,office,single_family,single_family,8423484308586815884,9146817053558612433,32.00,female,asian_not_hispanic_or_latino,employed,in_person,"17,261.00",private_auto,11.00,"318,660.00",three_plus,core,naics812199,single_family,not_attending_school,some_college,owner,asian_pacific,"1 (Tract 2623.01, Los Angeles, CA)","2623.01 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 7010, Los Angeles, CA)","7010 (Los Angeles, CA)","Los Angeles County, CA",California


In [70]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 9146817053558612433)>>count(_.trip_taker_person_id, _.trip_taker_age,
                                                                         _.trip_taker_sex, _.trip_taker_household_size)

,trip_taker_person_id,trip_taker_age,trip_taker_sex,trip_taker_household_size,n
0,4508016952300979367,25.00,female,11.00,5
1,4736398356956282212,38.00,female,11.00,2
2,8423484308586815884,32.00,female,11.00,2
3,12395012647665989305,21.00,female,11.00,1
4,14673592671547908751,18.00,female,11.00,4
5,16545634957747565398,49.00,female,11.00,1
6,16684115452596066816,53.00,male,11.00,1


In [71]:
display(HTML("<h4><strong>Number of Travelers by Resident Type: Replica Trips Data</strong></h4>")) 

In [72]:
r_trips>>group_by(_.trip_taker_resident_type)>>summarize(_unique_ids = _.trip_taker_person_id.nunique())

,trip_taker_resident_type,_unique_ids
0,core,246597
1,donut,2
2,visitor,35198
3,NaN,1


In [73]:
display(HTML("<h4><strong>Traveler Demographics: Replica Trips Data</strong></h4>")) 
display(HTML("Fitered for Core Residents")) 
(r_trips
 >>filter(_.trip_taker_home_bgrp_2020!="Visitor (no home location)")
 >>group_by(_.primary_mode, _.trip_taker_sex)
 >>summarize(
     n_ppl = _.trip_taker_person_id.nunique(),
     avg_h_income = _.trip_taker_household_income.mean(),
     avg_p_income = _.trip_taker_individual_income.mean(),
     avg_age = _.trip_taker_age.mean())
)
     

,primary_mode,trip_taker_sex,n_ppl,avg_h_income,avg_p_income,avg_age
0,auto_passenger,female,44266,"162,899.90","57,807.91",43.65
1,auto_passenger,male,43836,"173,439.46","87,237.70",42.86
2,commercial,NaN,1,NaN,NaN,NaN
3,on_demand_auto,female,1560,"181,325.52","58,599.19",46.58
4,on_demand_auto,male,1532,"181,278.14","92,035.74",43.23
5,private_auto,female,90634,"167,383.19","62,989.92",43.79
6,private_auto,male,99967,"174,477.16","94,470.93",43.81
7,private_auto,NaN,1,NaN,NaN,NaN
8,public_transit,female,927,"81,553.45","39,058.57",42.90
9,public_transit,male,979,"97,733.91","44,295.90",39.82
